In [1]:
pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import pymysql

In [3]:
#음악의 특성 값들의 평균을 계산
def calculate_average(cursor):
    cursor.execute("SELECT AVG(music_acousticness), AVG(music_danceability), AVG(music_energy), AVG(music_liveness), AVG(music_loudness), AVG(music_speechiness), AVG(music_tempo), AVG(music_valence) FROM mitiDB.album")
    return cursor.fetchone()

# 두 음악 간의 유사도를 구하는 함수
# 두 음악의 특성을 빼고 제곱을 더해서 제곱근을 구함. =유클리드 거리 계산 (수동 ㅎ)
def calculate_similarity(music_acousticness1, music_danceability1, music_energy1, music_liveness1, music_loudness1, music_speechiness1,
                          music_tempo1, music_valence1, music_acousticness2, music_danceability2,
                          music_energy2, music_liveness2, music_loudness2, music_speechiness2, music_tempo2, music_valence2):
    return np.sqrt((music_acousticness1 - music_acousticness2) ** 2 + (music_danceability1 - music_danceability2) 
                   ** 2 + (music_energy1 - music_energy2) ** 2 + (music_liveness1 - music_liveness2) ** 2 + (music_loudness1 - music_loudness2) 
                   ** 2 + (music_speechiness1 - music_speechiness2) ** 2 + (music_tempo1 - music_tempo2) ** 2) + (music_valence1 - music_valence2) ** 2 


#주어진 평균 음악 특성 값과 DB에 저장된 모든 음악의 특성을 비교하여 유사한 음악 찾음
def find_similar_music(cursor, avg_music_acousticness, avg_music_danceability, avg_music_energy, avg_music_liveness, avg_music_loudness,
                        avg_music_speechiness, avg_music_tempo, avg_music_valence):
    cursor.execute("SELECT id, music_acousticness, music_danceability, music_energy, music_liveness, music_loudness, music_speechiness, music_tempo, music_valence FROM mitiDB.album")
    music_list = cursor.fetchall() #결과를 misic_list에 저장
    
    similarities = []
    for music in music_list:
        id, music_acousticness, music_danceability, music_energy, music_liveness, music_loudness, music_speechiness, music_tempo, music_valence = music
        similarity = calculate_similarity(avg_music_acousticness, avg_music_danceability, avg_music_energy, avg_music_liveness, avg_music_loudness, avg_music_speechiness, avg_music_tempo, avg_music_valence, 
                                          music_acousticness, music_danceability, music_energy, music_liveness, music_loudness, music_speechiness, music_tempo, music_valence)
        similarities.append((id, similarity)) 
    
    # Sort by similarity (ascending order)
    similarities.sort(key=lambda x: x[1])  # 여기서는 similarity를 기준으로 정렬합니다.
    return similarities


In [6]:
def main():
    #mysql 연결
    conn = pymysql.connect(
    host = 'mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',
    user = 'minseo',
    password='Alstj!!809',
    db= 'mitiDB',
    charset= 'utf8'
    )

    #연결 잘 됐는지 select 해보며 확인
    cursor = conn.cursor()

    # 각 값 평균 계산
    avg_music_acousticness, avg_music_danceability, avg_music_energy, avg_music_liveness, avg_music_loudness, avg_music_speechiness, avg_music_tempo, avg_music_valence= calculate_average(cursor)
    # f-string을 사용하여 출력
    print(f'Average music_acousticness: {avg_music_acousticness}, Average music_danceability: {avg_music_danceability}, Average music_energy: {avg_music_energy}, Average music_liveness: {avg_music_liveness}, Average music_loudness: {avg_music_loudness}, Average music_speechiness: {avg_music_speechiness}, Average music_tempo: {avg_music_tempo}, Average music_valence: {avg_music_valence}')


    # 평균값이랑 유사한 음악 찾기
    similar_music = find_similar_music(cursor, avg_music_acousticness, avg_music_danceability, avg_music_energy, avg_music_liveness, avg_music_loudness, avg_music_speechiness, avg_music_tempo, avg_music_valence)
    
    # 비슷한 음악 10개
    print("Top 10 similar music:")
    for id, similarity in similar_music[:10]:
        print(f'ID: {id}, Similarity: {similarity}')



    # DB연결 종료
    conn.close()

if __name__ == "__main__":
    main()



Average music_acousticness: 0.32357630573248386, Average music_danceability: 0.5947961783439492, Average music_energy: 0.5306242038216561, Average music_liveness: 0.15507324840764336, Average music_loudness: -7.727929936305739, Average music_speechiness: 0.07969490445859874, Average music_tempo: 120.27943949044588, Average music_valence: 0.4098757961783439
Top 10 similar music:
ID: 52, Similarity: 1.2447886887331705
ID: 78, Similarity: 1.8742219299584235
ID: 117, Similarity: 2.3765345471663526
ID: 63, Similarity: 2.8918118853212516
ID: 94, Similarity: 3.315203552116657
ID: 111, Similarity: 3.869699669851518
ID: 19, Similarity: 4.221725526742655
ID: 132, Similarity: 4.461514780683055
ID: 79, Similarity: 4.502143723367986
ID: 4, Similarity: 5.021052167063233
